In [3]:
# Installation des nouvelles briques
#!pip install polars pyarrow

# Import Librairies

In [ ]:
import polars as pl
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin

# Pour conserver les noms de colonnes dans le pipeline
set_config(transform_output="pandas")

: 

# Fonction de Préparation des données

In [ ]:
class PolarsPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, skip_first_row=True):
        self.skip_first_row = skip_first_row

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X ici sera une liste de chemins [sirh, eval, sondage]
        sirh_path, eval_path, sondage_path = X
        
        # Votre logique Polars
        df_sirh = pl.read_csv(sirh_path)
        df_eval = pl.read_csv(eval_path)
        df_sondage = pl.read_csv(sondage_path)

        df_eval = df_eval.with_columns(
            pl.col("eval_number").str.replace("E_", "").cast(pl.Int64)
        )

        df_final = (
            df_sirh
            .join(df_eval, left_on="id_employee", right_on="eval_number", how="inner")
            .join(df_sondage, left_on="id_employee", right_on="code_sondage", how="inner")
        )

        df_final = df_final.with_columns([
            (pl.col("augementation_salaire_precedente")
             .str.replace(" %", "")
             .cast(pl.Float64) / 100).alias("augmentation_salaire_precedente_pourcentage")
        ])

        if self.skip_first_row:
            df_final = df_final.tail(-1)

        # On convertit en Pandas à la toute fin du transform pour Scikit-Learn
        return df_final.to_pandas()

: 

: 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1. Identification des colonnes
numeric_features = [
    'age', 'revenu_mensuel', 'annee_experience_totale', 
    'annees_dans_l_entreprise', 'distance_domicile_travail',
    'augmentation_salaire_precedente_pourcentage'
]

categorical_features = [
    'statut_marital', 'departement', 'poste', 
    'domaine_etude', 'frequence_deplacement', 'heure_supplementaires'
]

# 2. Création du preprocessor (C'est cette variable qui manquait !)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

: 

: 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

global_pipeline = Pipeline(steps=[
    ('polars_cleaner', PolarsPreprocessor(skip_first_row=True)), # Assurez-vous que la classe est définie
    ('sklearn_preprocessor', preprocessor), 
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

print("✅ Pipeline global créé avec succès !")

✅ Pipeline global créé avec succès !


: 

: 

In [ ]:
# 1. Le préprocesseur Polars (Nettoyage + Jointures)
# 2. Le ColumnTransformer (Standardisation + OneHot)
# 3. Le Classifieur

global_pipeline = Pipeline(steps=[
    ('polars_cleaner', PolarsPreprocessor(skip_first_row=True)),
    ('sklearn_preprocessor', preprocessor), # Votre ColumnTransformer déjà défini
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

: 

: 

# Test

In [ ]:
import polars as pl
import numpy as np
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

# --- 1. REDÉFINITION DE LA FONCTION DE BASE (pour y_train) ---
def prepare_data_polars(sirh_path, eval_path, sondage_path):
    df_sirh = pl.read_csv(sirh_path)
    df_eval = pl.read_csv(eval_path)
    df_sondage = pl.read_csv(sondage_path)

    # Nettoyage ID
    df_eval = df_eval.with_columns(
        pl.col("eval_number").str.replace("E_", "").cast(pl.Int64)
    )

    # Jointures
    df_final = (
        df_sirh
        .join(df_eval, left_on="id_employee", right_on="eval_number", how="inner")
        .join(df_sondage, left_on="id_employee", right_on="code_sondage", how="inner")
    )

    # Feature Engineering & Target
    df_final = df_final.with_columns([
        (pl.col("augementation_salaire_precedente")
         .str.replace(" %", "")
         .cast(pl.Float64) / 100).alias("augmentation_salaire_precedente_pourcentage"),
        
        pl.col("a_quitte_l_entreprise").map_elements(
            lambda x: 1 if x == "Oui" else 0, return_dtype=pl.Int64
        ).alias("target")
    ])

    return df_final.tail(-1)

# --- 2. EXÉCUTION DU TEST ---

# Définition des fichiers sources
raw_files = ['extrait_sirh.csv', 'extrait_eval.csv', 'extrait_sondage.csv']

try:
    # A. Préparation de la cible
    print("Étape 1: Préparation de la cible y...")
    data_full = prepare_data_polars(*raw_files)
    y_train = data_full["target"].to_numpy()

    # B. Entraînement du Pipeline Global
    print("Étape 2: Entraînement du pipeline global (Fit)...")
    global_pipeline.fit(raw_files, y_train)
    print("✅ Pipeline global entraîné avec succès !")

    # C. Test de prédiction
    print("Étape 3: Test de prédiction sur fichiers bruts...")
    predictions = global_pipeline.predict(raw_files)
    probabilities = global_pipeline.predict_proba(raw_files)[:, 1]

    # D. Affichage des résultats
    resultats = pl.DataFrame({
        "ID_Employe": data_full["id_employee"],
        "Prediction": predictions,
        "Probabilite_Depart": [f"{p*100:.1f}%" for p in probabilities]
    })

    print("\n--- RÉSULTATS DU TEST ---")
    print(resultats.head())

except Exception as e:
    print(f"❌ Erreur lors du test : {e}")
    print("Vérifiez que 'global_pipeline' et 'PolarsPreprocessor' sont bien définis au-dessus.")

Étape 1: Préparation de la cible y...
Étape 2: Entraînement du pipeline global (Fit)...
✅ Pipeline global entraîné avec succès !
Étape 3: Test de prédiction sur fichiers bruts...

--- RÉSULTATS DU TEST ---
shape: (5, 3)
┌────────────┬────────────┬────────────────────┐
│ ID_Employe ┆ Prediction ┆ Probabilite_Depart │
│ ---        ┆ ---        ┆ ---                │
│ i64        ┆ i64        ┆ str                │
╞════════════╪════════════╪════════════════════╡
│ 2          ┆ 0          ┆ 4.0%               │
│ 4          ┆ 1          ┆ 80.0%              │
│ 5          ┆ 0          ┆ 21.0%              │
│ 7          ┆ 0          ┆ 9.0%               │
│ 8          ┆ 0          ┆ 5.0%               │
└────────────┴────────────┴────────────────────┘


: 

: 

# Sauvegarder le pipeline complet (Preprocessing + Modèle)

In [ ]:
# Sauvegarde d'UN SEUL fichier
joblib.dump(global_pipeline, 'full_techNova_pipeline.pkl')

['full_techNova_pipeline.pkl']

: 

: 

# FastAPI

In [ ]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


: 

: 

: 

: 